<a href="https://colab.research.google.com/github/harperd17/energy_volatility_prediction/blob/main/merge_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal - Combine All Files in Merge Ready Folder -> Model Notebooks

In [ ]:
import pandas as pd
from google.colab import drive
import glob

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/Shareddrives/Data606_Energy/data/merge_ready

/content/drive/Shareddrives/Data606_Energy/data/merge_ready


In [ ]:
merge_files = glob.glob('*.csv')
merge_files

['annual_futures_summary.csv',
 'annual_weather_data.csv',
 'target_matrix_yearly_COV.csv',
 'annual_agg_powerplant_netgeneration.csv',
 'annual_2001_2021_electricity_sales_RES_COM.csv',
 'merged_data.csv',
 'annual_powerplant_netgeneration_2008_2021.csv',
 'annual_powerplant_netgeneration_2001_2021.csv',
 'merged_data_2001-2020.csv',
 'annual_agg_powerplant_netgeneration_2001_2021.csv']

In [ ]:
first_year = 2001 #because electric account data doesn't start until 2001
last_year = 2021
merged_data = None
for i, file in enumerate(merge_files):

  files_2008_2021 = ["annual_powerplant_netgeneration.csv", "annual_agg_powerplant_netgeneration.csv","annual_powerplant_netgeneration_2008_2021.csv"]
  if 'merged' not in file and file not in files_2008_2021:
    df = pd.read_csv(file)

    # Remove duplicate index
    if "Unnamed: 0" in df.columns:
         df = df.drop(columns="Unnamed: 0")

    #first, make sure year column is named consistently
    try:
      df['Year']
    except:
      df['Year'] = df['year'] # Currently failing here on annual_powerplant_netgeneration.csv
      del df['year']
    # now make sure state column is named consistently
    try:
      df['State']
    except:
      try:
        df['State'] = df['state']
        del df['state']
      except:
        ...# this is in the case that there isn't state in the dataframe - futures data doesn't have state for example
    # now subset the dataframe to only have data for the time window between first and last year
    df = df[(df['Year']>=first_year)&(df['Year']<=last_year)]
    # I'm doing a groupby so that indices will match up nicely for the joins
    # also because the electricity data is still at a monthly level, so averaging it seems appropriate to get it to annual
    if 'State' in df.columns:
      df = df.groupby(['Year','State']).mean()
    else:
      df = df.groupby(['Year']).mean()
    # now try to merge the data. If merge data doesn't exist yet then create it
    if merged_data is None:
      merged_data = df.copy()
    else:
      merged_data = pd.merge(merged_data,df,left_index=True, right_index=True)#on=['Year','State'])#,how='outer')
      
    if len([x for x in merged_data if "Unnamed:" in x]) != 0:
      print(f"{file} had unnamed data.")

In [ ]:
merged_data.head()

Months Above Average Price Movement BZ=F  ...  year.1
Year   State                                            ...        
2001.0 AK                                            0  ...     NaN
       AL                                            0  ...     NaN
       AR                                            0  ...     NaN
       AZ                                            0  ...     NaN
       CA                                            0  ...     NaN

[5 rows x 120 columns]

In [ ]:
merged_data.to_csv('merged_data_2001-2020.csv')